In [1]:
import os
import numpy as np
import pandas as pd
import holopy as hp
from PIL import Image
from src.utils.const import *

import matlab.engine
eng = matlab.engine.start_matlab()
s = eng.genpath(os.path.join(BASEPATH, 'matlab'))
eng.addpath(s, nargout=0)

df = pd.read_csv(datapath / Path('raw/indoor_objects.csv'))
df_dict = {k: (v1, v2) for k, (v1, v2) in zip(
    df['id'], zip(df['name'], df['classification']))}

In [ ]:
###############
# SPECIFICS
###############

locations = ['indoor', 'outdoor']
prefixes = {
    'indoor': ['in'],
    'outdoor': ['']
}
positions = {
    'indoor': ['bas', 'low', 'pad'],
    'outdoor': ['']
}
info = {
    'indoor':{
        'prefix': 'in',
        'indexes': list(range(0, 5)),
        'keys': ["location", "distance_from_source", "id", "orientation", "shape"],
        'columns': ['file_name', 'id', 'category', 'name', 'orientation', 'distance_from_source', 'inclination', 'shape', 'location']
    },
    'output':{
        'prefix': 'out',
        'indexes': list(range(0, 3)),
        'keys': ["id", "orientation", "shape"],
        'columns': ['file_name', 'id', 'category', 'orientation', 'shape', 'location', 'additional']
    }
}


In [ ]:
metadata = {
    'indoor': [],
    'outdoor': [],
}

In [ ]:
####################
# create HOLOGRAMS #
####################


for location in locations:
    
    names = os.listdir( os.path.join(datarawpath, location) )
    
    # create folder if not exists
    if not os.path.exists(os.path.join(holo_imagespath, location)):
        os.makedirs(os.path.join(holo_imagespath, location))  

    # instead of doing this:
    #   plutos = [name for name in names if name.endswith('.log')]
    #   sambas = [name for name in names if name.endswith('.csv')]
    # we just take the names without extention:
    names_list = [name.split('.')[0] for name in names]
    union = set(names_list)
    intersection = set([ el for el in union if names_list.count(el) > 1])

    indexes = info[location]['indexes']
    keys = info[location]['keys']
    prefix = info[location]['prefix']
    columns = info[location]['columns']

    diff = union - intersection

    for name in intersection:
        pluto = f'{name}.log'
        samba = f'{name}.csv'

        name_list = name.split('_')
        if len(name_list) > len(indexes):
            custom_indexes = np.ones(len(indexes), dtype=int)
            custom_indexes[0] = 0
            custom_indexes[1] = 0
            custom_indexes[2] = 0
            custom_indexes += indexes
            inclination = 20
        else:
            inclination = 0
            custom_indexes = indexes

        obj = {}
        for c_index, c_key in zip(custom_indexes, keys):
            obj[f'{prefix}_{c_key}'] = name_list[c_index]

        obj[f'{prefix}_location'] = location
        obj[f'{prefix}_distance_from_source'] = 8 if obj[f'{prefix}_distance_from_source'] == 'low' else 4 if obj[f'{prefix}_distance_from_source'] == 'bas' else None
        obj[f'{prefix}_inclination'] = inclination
        obj[f'{prefix}_file_name'] = name
        category_, name_ = df_dict.get(int(obj[f'{prefix}_id']), (None, None))
        # category = "mine"
        obj[f'{prefix}_category'] = category_
        # name = "pmn-4"
        obj[f'{prefix}_name'] = name_

        metadata[location].append(obj)

        for prefix in prefixes[location]:
            try:
                eng.workspace['pluto']= os.path.join(datarawpath, location, pluto)
                eng.workspace['trace']= os.path.join(datarawpath, location, samba)

                eng.eval(f"[F,FI,FQ,P_X,P_Y,P_MOD,P_PHASE] = merge_acquisition(trace, pluto);",nargout=0)
                eng.eval(f"[MO, PH, H, Hfill] = fast_generate_hologram(F, FI, FQ, 5, P_X, P_Y, P_MOD, P_PHASE, 2, 3);",nargout=0)

                #!################
                #! VARIABLES
                #!################
                # MO = eng.workspace['MO']
                # np_MO = np.asarray(MO, dtype = 'complex_')

                # PH = eng.workspace['PH']
                # np_PH = np.asarray(PH, dtype = 'complex_')

                # H = eng.workspace['H']
                # np_H = np.asarray(H, dtype = 'complex_')
                
                Hfill = eng.workspace['Hfill']
                np_Hfill = np.asarray(Hfill, dtype = 'complex_')
                
                # save image to file
                I8 = (((np_Hfill - np_Hfill.min()) / (np_Hfill.max() - np_Hfill.min())) * 255).astype(np.uint8)
                img = Image.fromarray(I8)
                img.save(os.path.join(holo_imagespath, location, f'{name}.png'))

            except Exception as e:
                print(e)

    df = pd.DataFrame.from_dict(metadata[location])
    
    columns = [f'{prefix}_{column}' for column in columns]
    df.to_csv(meta_imagespath / Path(f'{location}.csv'), index=False, header=True, columns=columns)
